# Master Discogs Database

In [19]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils, allmusicUtils
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from artistAM import artistAM
from artistsAM import artistsAM
from albums import albums
from album import album, albumURLInfo
from time import sleep

from masterdb import discConv, isKnownAlbum, directoryName
from masterdb import getSlimArtistDB, getArtistNameToIDMap
from masterdb import getMusicData, getRowByIndex

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-02-02 19:09:42.690780


In [2]:
%load_ext autoreload
%autoreload
discDC = discogs()
arts = artists(discDC)
art  = artist()
albs = albums(discDC)
alb  = album()
dutils = discogsUtils()

discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Warning! Saved Discog Directory None is not Available
Warning! Saved Discog Directory None is not Available


# Create Discogs <-> AllMusic Map

In [3]:
discSlimArtistDCDB = getSlimArtistDB(discDC, force=False)
artistDCNameToID   = getArtistNameToIDMap(discSlimArtistDCDB)
artistDCKeys       = artistDCNameToID.keys()

Current Time is Sun Feb 02, 2020 17:47:19 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Current Time is Sun Feb 02, 2020 17:47:21 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 2 seconds.
Current Time is Sun Feb 02, 2020 17:47:21 for 
=================================== Creating Artist Name To ID Map ===================================
Found 712629 ID -> Name entries
Found 654970 Name -> ID entries


In [4]:
discSlimArtistAMDB = getSlimArtistDB(discAM, force=False)
artistAMNameToID   = getArtistNameToIDMap(discSlimArtistAMDB)
artistAMKeys       = artistAMNameToID.keys()

Current Time is Sun Feb 02, 2020 17:47:22 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Current Time is Sun Feb 02, 2020 17:47:22 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 132.0 millseconds.
Current Time is Sun Feb 02, 2020 17:47:22 for 
=================================== Creating Artist Name To ID Map ===================================
Found 36432 ID -> Name entries
Found 34373 Name -> ID entries


In [5]:
artistAMToAlbumsData = discAM.getArtistIDToAlbumNamesData(debug=True)
print(len(artistAMToAlbumsData))

Data stored in /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
  --> This file is 11.8MB.
Loading /Users/tgadfort/Music/Discog/db-allmusic/ArtistIDToAlbumNames.p
36432


In [6]:
artistDCToAlbumsData = discDC.getArtistIDToAlbumNamesData(debug=True)
print(len(artistDCToAlbumsData))

Data stored in /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
Returning data from /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
Loading data from /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToAlbumNames.p
712629


In [23]:
from searchUtils import findNearest
num    = 3
cutoff = 0.85

In [ ]:
flat_list = [item for sublist in l for item in sublist]

In [11]:
allMusicToCheck = getFile(ifile="/Users/tgadfort/Documents/code/music/allmusicToCheck.p")

In [12]:
for musicName in allMusicToCheck:
    

['A-Mafia',
 'Admiral D',
 'Adryan Russ',
 'Afrodite',
 'Aftertaste',
 'Age Of Love',
 'Agent 99',
 'AIR',
 'Alberte',
 'Alley Boy',
 'Amy Rigby',
 'Andy G And The Roller Kings',
 'Apothys',
 'Arkham 13',
 'Assück',
 'Ayahuasca',
 'Bad Times',
 'Balaam And The Angel',
 'Barry Berrier',
 'Barry Kleinbort',
 'Bass',
 'Beam & Yanou',
 'Beat Guru',
 'Beatrice Arthur',
 'Beavis & Butthead',
 'Bebe Rebozo',
 'Bedřich Smetana',
 'Before I Break',
 'Bei Maejor',
 'Bells Of Avalon',
 'Benett Rogers',
 'Benümb',
 'Bert Jansch And John Renbourn',
 'Beth Capper',
 'Betty Comden And Adolph Green',
 'Billy Dixon & The Topics',
 'Black Cobain',
 'Blastula',
 'Blazing Rains',
 'Blue Ridge Mountain',
 'Blyss',
 'Bo Deal',
 'Bombario',
 'Bottom Feeders',
 'Bounty Killaz',
 'Boysetsfire',
 'Brandon Cruz',
 'Brazil 2001',
 'Bruce Hampton',
 'Brutal DeLuxe',
 'Bryan Adams',
 'Buick',
 'Burning Witch',
 'The',
 'Caddywompas',
 'Cambatta',
 'Capital 2',
 'Cee-Lo',
 'Charles Kuralt',
 'Charm Pops',
 "Children

In [7]:
artistAMIDtoDCIDMap = getFile(ifile="artistAMIDtoDCIDMap.p")
artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

In [25]:
print(len(artistAMNameToDCNamesMap))

print(len(allMusicToCheck))
for iKey,artistAMName in enumerate(allMusicToCheck):
    if artistAMNametoIDMap.get(artistAMName):
        continue
    if artistAMName in artistDCKeys:
        artistAMNameToDCNamesMap[artistAMName] = [artistAMName]
        
    if True:
        artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
        if len(artistDCNames) > 0:
            print(iKey,'/',len(allMusicToCheck),'\t',artistAMName)
            artistAMNameToDCNamesMap[artistAMName] = artistDCNames
            
print(len(artistAMNameToDCNamesMap))

19525
687
0 / 687 	 A-Mafia
1 / 687 	 Admiral D
2 / 687 	 Adryan Russ
3 / 687 	 Afrodite
4 / 687 	 Aftertaste
5 / 687 	 Age Of Love
6 / 687 	 Agent 99
7 / 687 	 AIR
8 / 687 	 Alberte
9 / 687 	 Alley Boy
10 / 687 	 Amy Rigby
11 / 687 	 Andy G And The Roller Kings
12 / 687 	 Apothys
13 / 687 	 Arkham 13
14 / 687 	 Assück
15 / 687 	 Ayahuasca
16 / 687 	 Bad Times
17 / 687 	 Balaam And The Angel
18 / 687 	 Barry Berrier
19 / 687 	 Barry Kleinbort
20 / 687 	 Bass
21 / 687 	 Beam & Yanou
22 / 687 	 Beat Guru
23 / 687 	 Beatrice Arthur
24 / 687 	 Beavis & Butthead
25 / 687 	 Bebe Rebozo
26 / 687 	 Bedřich Smetana
27 / 687 	 Before I Break
28 / 687 	 Bei Maejor
29 / 687 	 Bells Of Avalon
30 / 687 	 Benett Rogers
31 / 687 	 Benümb
32 / 687 	 Bert Jansch And John Renbourn
33 / 687 	 Beth Capper
34 / 687 	 Betty Comden And Adolph Green
35 / 687 	 Billy Dixon & The Topics
36 / 687 	 Black Cobain
37 / 687 	 Blastula
38 / 687 	 Blazing Rains
39 / 687 	 Blue Ridge Mountain
40 / 687 	 Blyss
41 / 687 	

337 / 687 	 Superstar Jay
338 / 687 	 Svasti-ayanam
339 / 687 	 Sønderjyllands Symfoniorkester
340 / 687 	 Søren Kragh-Jacobsen
341 / 687 	 Team Invasion
342 / 687 	 Tears For Fears
343 / 687 	 Tee And Thee Crumpets
344 / 687 	 Tee Stunna
345 / 687 	 Tertium Non Data
346 / 687 	 Tédio Boys
347 / 687 	 Thermocline
348 / 687 	 These 5 Down
349 / 687 	 Theta Wave State
350 / 687 	 Thinking Man
351 / 687 	 This World Rejected
352 / 687 	 Throw That Beat In The Garbagecan!
353 / 687 	 Thumbs Down
354 / 687 	 Tiesto
355 / 687 	 Tom Petty And The Heartbreakers
356 / 687 	 Tommy Malone
357 / 687 	 Tone Loc
358 / 687 	 Tony De Vit
359 / 687 	 Top Gun
360 / 687 	 Trail Of Tears
361 / 687 	 Tribal Lust And The Horny Natives
362 / 687 	 Trust No One
363 / 687 	 Tube Top
364 / 687 	 Tuna Universitaria de Granada
365 / 687 	 Twin Princess
366 / 687 	 The Allstonians
367 / 687 	 The American Fuse
368 / 687 	 The Anomoanon
369 / 687 	 The Aqua Velvets
370 / 687 	 The Arkay IV
371 / 687 	 The B. Lee Ba

In [16]:
#artistAMNameToDCNamesMap = {}

print(len(artistAMNameToDCNamesMap))

print(len(artistAMKeys))
for iKey,artistAMName in enumerate(artistAMKeys):
    if iKey <= 14386:
        continue
    if artistAMNametoIDMap.get(artistAMName):
        continue
    if artistAMName in artistDCKeys:
        continue
        artistAMNameToDCNamesMap[artistAMName] = [artistAMName]
        
    if True:
        artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
        if len(artistDCNames) > 0:
            print(iKey,'/',len(artistAMKeys),'\t',artistAMName)
            artistAMNameToDCNamesMap[artistAMName] = artistDCNames
            
print(len(artistAMNameToDCNamesMap))

19525
34373


NameError: name 'findNearest' is not defined

In [8]:
#saveFile(idata=artistAMNameToDCNamesMap, ifile="artistAMNameToDCNamesMap.p")
artistAMNameToDCNamesMap = getFile(ifile="artistAMNameToDCNamesMap.p")

In [27]:
from difflib import SequenceMatcher
import operator

def getArtistAlbums(vals):
    albums = []
    for k,v in vals.items():
        for k2, v2 in v.items():
            albums.append(v2)
    return albums
            
    
def getBestAlbumsMatch(albumsAM, albumsDC, debug=False):
    score = 0.0
    for albumAMAlbum in albumsAM:
        for albumDCAlbum in albumsDC:
            s = SequenceMatcher(None, albumAMAlbum, albumDCAlbum)
            ratio = round(s.ratio(), 3)
            if debug:
                print("\t\t\t[{0}] + [{1}] = [{2}]".format(albumAMAlbum, albumDCAlbum, ratio))
            if ratio > 0.7:
                score += ratio
    return score

def showNames(artistAMName, artistDCNames):
    print("{0: <50}{1}".format(artistAMName, artistDCNames))



artistAMIDtoDCIDMap = getFile(ifile="artistAMIDtoDCIDMap.p")
artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

for iKey,artistAMName in enumerate(artistAMKeys):
    
    if artistAMNametoIDMap.get(artistAMName):
        continue
    artistDCNames = artistAMNameToDCNamesMap.get(artistAMName)
    if artistDCNames is None:
        continue
    #artistDCNames = findNearest(artistAMName, artistDCKeys, num=3, cutoff=cutoff)
    #if len(artistDCNames) == 0:
    #    continue
    
    artistAMIDs = artistAMNameToID[artistAMName]
    
    ## Loop over Artist AM IDs
    for iAMID,artistAMID in enumerate(artistAMIDs):
        if artistAMIDtoDCIDMap.get(artistAMID) is not None:
            continue
        
        showNames(artistAMName, artistDCNames)
        artistAMAlbums = getArtistAlbums(artistAMToAlbumsData.get(artistAMID))
        if artistAMAlbums is None:
            print("\tAM ID={0: <15}{1: <5}/{2: <8}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, "NONE"))
        else:
            print("\tAM ID={0: <15}{1: <5}/{2: <8}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, len(artistAMAlbums)))

            
        artistAMtoDCScore = None
        
            
        ## Loop over Artist DC Names
        for artistDCName in artistDCNames:
            artistDCIDs = artistDCNameToID[artistDCName]
            
            ## Loop over Artist DC IDs
            for artistDCID in artistDCIDs:
                artistDCAlbums = getArtistAlbums(artistDCToAlbumsData.get(artistDCID))
                if False:
                    if artistDCAlbums is None:
                        print("\t\tDC ID={0: <15}\t      {1}".format(artistDCID, "NONE"))
                    else:
                        print("\t\tDC ID={0: <15}\t Size={1}".format(artistDCID, len(artistDCAlbums)))
                
                retval = getBestAlbumsMatch(artistAMAlbums, artistDCAlbums, debug=False)
                if retval > 0.7:
                    if artistAMtoDCScore is None:
                        artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                    else:
                        if retval > artistAMtoDCScore["Score"]:
                            artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                            
                            
        print("\tAM ID={0: <15}{1}".format(artistAMID, artistAMtoDCScore))
        if artistAMtoDCScore is not None:
            artistAMIDtoDCIDMap[artistAMID] = {"AM Name": artistAMName, "DC ID": artistAMtoDCScore["ID"], "DC Name": artistAMtoDCScore["Name"]}
            print("Found Match for {0} --> {1}".format(artistAMID, artistAMIDtoDCIDMap[artistAMID]))
            saveFile(idata=artistAMIDtoDCIDMap, ifile="artistAMIDtoDCIDMap.p")
            print("\n")
        


Bebe Rebozo                                       ['Bebe Rebozo']
	AM ID=0001617506     9    /34373   0    1
	AM ID=0001617506     None
Jennifer Jones                                    ['Jennifer Johns', 'Jennifer John', 'Jennifer Johnson']
	AM ID=0000315406     15   /34373   0    1
	AM ID=0000315406     None
Cosmic Passengers                                 ['Cosmic Passengers']
	AM ID=0002632906     17   /34373   0    1
	AM ID=0002632906     None
Bambir                                            ['Bambi']
	AM ID=0000396306     26   /34373   0    2
	AM ID=0000396306     None
Willie White                                      ['Willie Whyte', 'Will White', 'William White']
	AM ID=0001658906     39   /34373   0    2
	AM ID=0001658906     None
The Young Immortals                               ['The Young Immortals']
	AM ID=0001567806     55   /34373   0    3
	AM ID=0001567806     None
MARS                                              ['MRS', 'MAR']
	AM ID=0002888206     66   /34373   0  

	AM ID=0003413994     None
Christoph Schiller                                ['Christoph Schiller']
	AM ID=0001641594     415  /34373   0    1
	AM ID=0001641594     None
Fuel On Fire                                      ['Fuel On Fire']
	AM ID=0003893094     416  /34373   0    1
	AM ID=0003893094     None
Intro to Airlift                                  ['Intro To Airlift']
	AM ID=0000477794     417  /34373   0    1
	AM ID=0000477794     None
Lazlo Bane                                        ['Laszlo Band']
	AM ID=0000787194     431  /34373   0    5
	AM ID=0000787194     None
dvsn                                              ['dvsn']
	AM ID=0003488294     433  /34373   0    14
	AM ID=0003488294     None
Tamouz                                            ['Tamouz']
	AM ID=0001533194     437  /34373   0    2
	AM ID=0001533194     None
Rosa Henderson                                    ['Roy Henderson', 'Russ Henderson']
	AM ID=0000338394     451  /34373   0    6
	AM ID=0000338394     None

	AM ID=0002227640     None
Casualty                                          ['Casual']
	AM ID=0001626440     1437 /34373   0    2
	AM ID=0001626440     None
Jr Lonestar                                       ['DJ Lonestar']
	AM ID=0003708115     1461 /34373   0    1
	AM ID=0003708115     None
Beavis & Butt-Head                                ['Beavis & Butt-Head']
	AM ID=0000133415     1470 /34373   0    6
	AM ID=0000133415     None
Bad Boyz DJ Team                                  ['Bad Boyz DJ Team']
	AM ID=0000431315     1485 /34373   0    1
	AM ID=0000431315     None
Hotshot                                           ['Hotshot']
	AM ID=0001514015     1491 /34373   0    1
	AM ID=0001514015     None
Hotshot                                           ['Hotshot']
	AM ID=0002924356     1491 /34373   1    3
	AM ID=0002924356     None
Hotshot                                           ['Hotshot']
	AM ID=0003358174     1491 /34373   2    1
	AM ID=0003358174     None
Trey Spruance             

	AM ID=0001410222     None
Headshots                                         ['Headshots']
	AM ID=0001810222     2032 /34373   0    1
	AM ID=0001810222     None
bis                                               ['Ibis']
	AM ID=0000099422     2035 /34373   0    26
	AM ID=0000099422     None
Vienna Scientists                                 ['Vienna Scientists']
	AM ID=0000537822     2038 /34373   0    1
	AM ID=0000537822     None
John Davison                                      ['John Davison']
	AM ID=0002163522     2041 /34373   0    1
	AM ID=0002163522     None
L'Arc-en-Ciel                                     ["L'Arc-En-Ciel"]
	AM ID=0000635822     2077 /34373   0    54
	AM ID=0000635822     None
Celebrity Drug Disasters                          ['Celebrity Drug Disasters']
	AM ID=0001529722     2087 /34373   0    1
	AM ID=0001529722     None
Inhumane                                          ['Inhumane']
	AM ID=0000436622     2105 /34373   0    1
	AM ID=0000436622     None
The Nod  

	AM ID=0000196499     None
Unit 4+2                                          ['Unit 4', 'Unit 2']
	AM ID=0000179499     2655 /34373   0    6
	AM ID=0000179499     None
Fly                                               ['Fly']
	AM ID=0000195899     2656 /34373   0    9
	AM ID=0000195899     None
Article Nine                                      ['Article One']
	AM ID=0003222699     2673 /34373   0    1
	AM ID=0003222699     None
Ost                                               ['Ost']
	AM ID=0000418699     2674 /34373   0    14
	AM ID=0000418699     None
Ost                                               ['Ost']
	AM ID=0001015573     2674 /34373   1    2
	AM ID=0001015573     None
The War Within                                    ['The War Within']
	AM ID=0001602599     2677 /34373   0    3
	AM ID=0001602599     None
Carpenter                                         ['Carpenters']
	AM ID=0001496099     2687 /34373   0    2
	AM ID=0001496099     None
Tucker Mountain Band                 

...

	AM ID=0001378916     None
Lloyd Price & His Orchestra                       ['Lloyd Price And His Orchestra', 'Lloyd Price Orchestra']
	AM ID=0001623616     31384/34373   0    1
	AM ID=0001623616     None
Mike Price                                        ['Mike Price']
	AM ID=0000101016     31387/34373   0    1
	AM ID=0000101016     None
Vitalij Margulis                                  ['Vitaly Margulis']
	AM ID=0001980605     31392/34373   0    7
	AM ID=0001980605     None
Popguns                                           ['Popgun']
	AM ID=0000488005     31396/34373   0    7
	AM ID=0000488005     None
Eagles Prey                                       ['Eagles Prey']
	AM ID=0002179105     31411/34373   0    1
	AM ID=0002179105     None
Kerry Engle                                       ['Jerry Engler']
	AM ID=0003302505     31432/34373   0    2
	AM ID=0003302505     None
Semtex                                            ['Semtex']
	AM ID=0002118705     31438/34373   0    2
	AM ID=0002

	AM ID=0000659368     None
Chris McKay                                       ['Chris McKay']
	AM ID=0001229668     32088/34373   0    2
	AM ID=0001229668     None
Ogre You Asshole                                  ['Ogre You Asshole']
	AM ID=0001578868     32101/34373   0    1
	AM ID=0001578868     None
Ken Kunin                                         ['Ken Kunin']
	AM ID=0002286568     32102/34373   0    1
	AM ID=0002286568     None
The Gamblers                                      ['The Gamblers']
	AM ID=0000063768     32107/34373   0    2
	AM ID=0000063768     None
The Works                                         ['The Works']
	AM ID=0001893968     32109/34373   0    2
	AM ID=0001893968     None
The Fabulous Four Tops                            ['The Fabulous Four']
	AM ID=0001309868     32116/34373   0    1
	AM ID=0001309868     None
Biggie Smalls                                     ['Biggie Smalls']
	AM ID=0001254368     32125/34373   0    1
	AM ID=0001254368     None
Beat Cultur

  --> This file is 697.6kB.


Los Bestias                                       ['Los Bestiales']
	AM ID=0000290686     32829/34373   0    1
	AM ID=0000290686     None
The Dream Weavers                                 ['The Dream Weavers']
	AM ID=0000137986     32835/34373   0    1
	AM ID=0000137986     None
Adam Ryan                                         ['Adam Ayan']
	AM ID=0002327986     32841/34373   0    1
	AM ID=0002327986     None
Kay Cee                                           ['Kay Cee', 'Kay Jee', 'Kay Gee']
	AM ID=0000857286     32844/34373   0    23
	AM ID=0000857286     None
Faster Disaster                                   ['Flaster Of Disaster']
	AM ID=0000436086     32847/34373   0    2
	AM ID=0000436086     None
Babylon Robots                                    ['Baby Robots']
	AM ID=0000601086     32867/34373   0    1
	AM ID=0000601086     None
Aurelio de la Vega                                ['Aurelio De La Vega']
	AM ID=0001784886     32868/34373   0    1
	AM I

	AM ID=0002184565     None
Milky Way                                         ['MilkyWay', 'Mikey Way']
	AM ID=0000506465     33528/34373   0    2
	AM ID=0000506465     None
Milton Brown                                      ['Milton Brown']
	AM ID=0000416165     33544/34373   0    13
	AM ID=0000416165     None
Note Native                                       ['New Native']
	AM ID=0002612865     33549/34373   0    1
	AM ID=0002612865     None
Russell Leon                                      ['Russell Leonce', 'Russell DeLeon', 'Russell Lee']
	AM ID=0002132165     33564/34373   0    1
	AM ID=0002132165     None
Poppy                                             ['Poppy']
	AM ID=0003667565     33565/34373   0    7
	AM ID=0003667565     None
Pure Blyss                                        ['Pure Blyss']
	AM ID=0001895765     33580/34373   0    1
	AM ID=0001895765     None
GONG                                              ['GOG', 'GNG']
	AM ID=0003807065     33603/34373   0    1
	AM ID=00

	AM ID=0001459708     None
Family K                                          ['Family 5', 'Family']
	AM ID=0003185408     34339/34373   0    1
	AM ID=0003185408     None
The Rich Kids                                     ['The Rich Kids']
	AM ID=0000342908     34347/34373   0    3
	AM ID=0000342908     None
John Gale                                         ['John Gale']
	AM ID=0000101708     34364/34373   0    1
	AM ID=0000101708     None
Blue Fox Knights                                  ['Blue Knights']
	AM ID=0002756308     34365/34373   0    2
	AM ID=0002756308     None
Shifter X                                         ['Shifter']
	AM ID=0002888808     34366/34373   0    1
	AM ID=0002888808     None


In [15]:
len(artistAMIDtoDCIDMap)

20072

# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

#### Dictionaries

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(discAM)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(disc)

#### Pandas DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistAMDB = getSlimArtistDB(discAM, force=True)
discSlimArtistAMDB.head()

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc, force=True)
discSlimArtistDB.head()

## Artist ID --> Albums

#### Dictionaries

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAM)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(disc)

#### Pandas DB

In [ ]:
from masterdb import getArtistAlbumsDB
discArtistAMAlbumsDB = getArtistAlbumsDB(discAM, force=True)
discArtistAMAlbumsDB.head()

In [ ]:
from masterdb import getArtistAlbumsDB
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=True)
discArtistAlbumsDB.head()

## Artist ID --> Genre, Style, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(disc)

## Album ID --> Name, Ref, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [ ]:
from masterdb import getArtistDB

In [ ]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

## Artist Metadata DB

In [ ]:
from masterdb import getArtistMetadataDB

In [ ]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

## Artist Albums DB

In [ ]:
from masterdb import getArtistAlbumsDB

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

## Albums DB

In [ ]:
from masterdb import getAlbumDB

In [ ]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

## Artist Album ID --> Known Albums

In [ ]:
from masterdb import getArtistAlbumKnownDB

In [ ]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random